# Project Title
### Data Engineering Capstone Project

#### Project Summary
Project to acquiring data from provided-workspace and extract information about: Immigration to US; Cities in US ; Airport in US.
Project intend to make a target tables for serving info-query in future (such as Security Investigate for people who immigrated to or leaved out US), by doing proper ETL job base on above datasets.
Workspace supplied also the world-Temprature data, this will be optional-work for attach more climate information or coordinates by cities.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets using for project: use supplied data by Udacity workspace.

##### Immigration Datasets:
* Comes from 12 file with format sas7bdat files.
* This data supply main information about immigration activities to US, that was acquired in all of months 2016.
* dataset will be used as main source to do ETL job for get target table for monitor who immigrated and leaved out US in 2016.

##### Airport dataset:
* The dataset provide information for airports on the world with some detail info such as : Airport Name, country, municipality (could understand as city in this project), coordinate of each one
* From municipality and US-State code suppplied in it, they should come to make joining data later.

##### US-cities demographic:
* The file provide detail information of each US cities: some information should support joining data such as State Code and City.

##### GlobalLandTemperatureByCity:
* This data come with more than 1 millions rows consists detail average temperature by city, expand on very long range of record time.
* This data will using when do join with AirPortCode and CityName, that can make a useful table for query climate information and location also for each city in specified time.

##### [NOTE-1] The step for read and acquiring data from above datasets have done in etl.py
##### [NOTE-2] The evidence for project includes datasets requires (at least 2 data sources; more than 1 millions line of data; at leaset two data sources/ format).
* The number of rows of Temperature data is >8 million lines
* The number of rows of Immigration data is >3 million lines per one file (totally is 12 immigration data files)
* Above information can be found in console result when test run etl.py
* In `./project_docs/dataset_evidence.pdf` has contains the evidence as mentions above
* In `./project_docs/dataset_evidence.pdf` also contains the SQL demo query result for `to group count immigrants by their cities and include the "foreign_born" field in this city`.

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

##### By Doing read datasets mentioned in Step1 either pandas read df or PySpark read Df, could be explore more about each dataset.
(attached pdf file: `./project_docs/data_explore.pdf`).

##### Immigration Datasets: 
(Please note: these observe below is for demo by one sas data file, for save time run test ETL; Whole datasets always can be read and acquired, as long as cleaningData function have subsetList for user use for droping NaN or Duplicated value during any time run ETL)
* By using the api to check NaN for each schema, found some point:
* Some important column of this data sets does not include any NaN value: cicid; arrdate; i94port; i94visa; or visatype
* The columns does not contain NaN values are important to consider choose vale for query data, or making Join with another dataset.

##### US-Cities Demogrphic:
* This dataset is the best in Nan consisting, it fews columns have NaN , with maximum Nan values is 16. Almost no need to care these NaN value in this project.
* For Duplicated value: as immagine use-case for Security investigation base on Immigration AirPort base on CityName, so the duplicated `City` will be consider to dropping during data cleaning.
         However, need to be considered number of cityname would be dropped.

##### Airport-Code:
* This dataset is quite good quality, important columns such as: ident, type, name, iso_region and coordinates do not contain any NaN value. This will be very good for checking location of airport in future. Another columns contain NaN ,it depend on user want to drop or not.

##### Temperature dataset:
* This dataset contains a lot missing value, especcially Average Temperature data.
* Duplicated value: Just handling during data cleaning step.

#### Cleaning Steps
Steps necessary to clean the data on each dataset:

##### Immigration Datasets: 
* In this poject:use the i94port for extract AirPortCode base on CityName (use by the sas_labels_descriptionfile), this airport code and cityname may not exist all in another dataset. However in Immigration Datasets, i94port not contain any NaN: that mean any case immigration or leave Us can be tracked by AirPort, (and there a number US cityname can be found from that)
* Some columns cotain a lot of NaN such as: gender/visapost/ entdepd. However, user can consider the target table using purpose to make drop the NaN on that table or not.
* For Duplicated values: not impact a lot during ETL, just add subset while drop such as (`cicid`): just for make sure not duplicated the Immigration information for same one person on same datetime.
* Another note: `arrdate` and `depdate` are provided as `timestamp` type, during loading data to table, it need to transform the ts to iso format type for Date and Time.
     
##### US-Cities Demogrphic:
* No need to care these NaN value in this project.
* For Duplicated value: as immagine use-case for Security investigation base on Immigration AirPort base on CityName, so the duplicated `City` will be consider to dropping during data cleaning. However, need to be considered number of cityname would be dropped.
    
##### Airport-Code:
* Columns contain NaN ,it depend on user want to drop or not, in this project not care about that NaN.
* Duplicated value: should care about `ident` and `name`: just for make sure in target table do not have duplicated same airport during checking.
    
##### Temperature dataset:
* They will be droped NaN value of AvgTemperatures during data cleaning. Because in same city, have plenty of temperature data was recorded.
* Duplicated value: do drop the `city`: just for keeping only one city contain temperature data, another important info as Coordinates is always be same for a city.
    
##### Note: 
* Cleaning job: is located as dedicated py module `schemas_assist.DataClean`
* In the file DataClean do data cleaning for all dataset type:
 * for each data set, data cleaning function could offer either subset list for NaN or for duplicated or both of them.
 * another extra-job in this file is: extract Airport Code + City Name from `SAS_Labels_Description file`. This combined information will work as a key for two dimension table Us-Citiies/ WorldTemp and AirPort; and all of them can be join to Immigration data set for query work later.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Project choose the data model Star Schema, reason to choose this:
* Dataset need to handle is quite large (each file sasdata more than 3mils rows, Temp > 1mils rows).
* And the dataset might be scale up in future, as well as the table need to be scale al so
* Need to leverage some tool for speed up staging and writing for big amount data size: Ex.using Pyspark DataFrame.

#### 3.2 Mapping Out Data Pipelines
Steps necessary to pipeline the data into the chosen data model:
* Using PySpark read data set with some different file format: csv, sas7bdat.
* Each time finish read a file, do read on Schema with api printSchema()
* Loding data from above staging Schema in to the target table (for Ex. Dimensional Airport table).
* Veriry the content inside table just created above
* Write table to parquet file:
   * consider choosing properly parameter for partitionBy() : for speed up writing and reading parquet files
   * choosing location to save parquet file: in real world, it should choosed AWS S3 bucket. For run test of this project, it still use local saving.

#### Conceptual Data Model:
The StarSchema was descripe in Table.xlsx file in `./project_docs/Table.xlsx`, please kindly find out in that

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.
 - Implemented in etl.py

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks: it locate in etl.py also

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.


Data dictionary will be included by Table.xlsx file in the `./project_docs/Table.xlsx`


#### Step 5: Complete Project Write Up
##### Why choice PySpark for handling datasets
* Data sets provied with large number of rows (millions of rows in two datasets)
* To handling, Join Data ... PySpark obviously better choice than pandas df.
* We can leverage the operation as dealing SQL command when need to Join, Select, Aggre with df by Spark
* Data will be scale up more and more in future , especially Imiggration Data. We need a tools, technologies for support huge computing, storaging resource. In real world, it's absoluty need to Cloud services (Ex. AWS: S3, Redshift, EC2)
* When do ETL with Spark: with Staging and Loading task, that we can choose either local or Cloud servie (like S3, Redshift, EC2).

##### Propose how often the data should be updated and why.
* The main data set for serving immigration information checking is Immigration Data. This is really big amount size.
* The second big dataset is City Temperature Data in whole world, this will scale up very fast due to time interval to get climate data.
    * For this type of data, prior to update Temeperature data to table, it should be care for cleaning data.
* For keep update information: interval time should be 1 week to month that depend on require.
* Reason:
    * For keep information up to date, it should be often update new info added, however, with this size of datasets, it will consume tons of resource and time.
    * So for high require information update: it should be update weekle, with another with lower demand: it can be update with interval 2weeks to 1 month.
    
##### Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     * If data come more really big, it totally need to use Cloud Computing service: we have two choice AWS service for working with Spark: AWS EMR or AWS EC2:
         * Both of them could be able to suppport Distributed computing and support HDFS
         * AWS EMR use S3 protocol faster than S3a which applied on AWS EC2
         * Basically we can choose either of them, with notice AWS EMR will be higer cost a bit.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.:
     * It suggest to use AirFlow.
     * AirFlow with strongest to make schedule for data pipeline execution
 * The database needed to be accessed by 100+ people.
     * Still come to AirFlow and AWS, with high demand of access data from user, we can:
     * Accomplish data from some where locate in S3, load for analythic like Redshift , and loading back database storage location.
     * All the ETL with big size amount should done by Cloud computing resource. AirFlow will monitor the flow and schedule for ETL jobs.